<a href="https://colab.research.google.com/github/songqsh/foo1/blob/master/src/mdp_1d_qtable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import time
start_time = time.time()
from tqdm import tqdm

In [0]:
#configure

TERM = 5

UP_LIM = 3
DN_LIM = -3

time_space = np.arange(TERM+1)
state_space = np.arange(DN_LIM, UP_LIM+1)
action_space = np.array([-1,1])


In [0]:
#configure

def terminal_cost(state):
  return state**2

def running_cost(time, state, action):
  return 0



In [0]:
#one step move

step_space = np.arange(-1, 2)

def next_state(state, step):
  state = state + step
  if state > UP_LIM:
    state = UP_LIM
  elif state < DN_LIM:
    state = DN_LIM
  return state
      
def next_state_ind(state_ind, step_ind):
  next_state_ind = state_ind + step_space[step_ind]
  if next_state_ind > state_space.size - 1:
    next_state_ind = state_space.size - 1
  elif next_state_ind < 0:
    next_state_ind = 0
  
  return next_state_ind

In [0]:
#transition probability

def transition_probability(time, state, action):
  tp = np.zeros(step_space.shape)
  if action == 1:
    tp = np.array([0, .5, .5])
  else:
    tp = np.array([.5, .5, 0])
    
  return tp


In [0]:
#q-table state value initialization
s_val = np.zeros([time_space.size, state_space.size])
q_table = np.zeros([time_space.size, state_space.size, action_space.size])

In [0]:
#q-table terminal setup
for i in range(state_space.size):
  for j in range(action_space.size):
    q_table[-1,i,j] = terminal_cost(state_space[i])
    

In [0]:
#sync q_table, s_val
def q_s_sync(t_ind, s_val, q_table):
  for i in range(state_space.size):
    s_val[t_ind, i] = np.min(q_table[t_ind,i])
    
q_s_sync(-1, s_val, q_table)

In [0]:
#backward iteration
for i in np.arange(time_space.size-2, -1, -1):
  for j in np.arange(state_space.size):
    for k in np.arange(action_space.size):
      t_ = time_space[i]
      x_ = state_space[j]
      a_ = action_space[k]
      
      cost_ = running_cost(t_, x_, a_)
      tp_ = transition_probability(t_, x_, a_)
      for ii in range(step_space.size):
        next_j = next_state_ind(j, ii)
        cost_ += tp_[ii]*s_val[i+1, next_j]
      q_table[i,j,k] = cost_
      
  q_s_sync(i, s_val, q_table)
  
      

In [15]:
s_val

array([[1.375, 0.625, 0.5  , 0.5  , 0.5  , 0.625, 1.375],
       [2.   , 0.75 , 0.5  , 0.5  , 0.5  , 0.75 , 2.   ],
       [3.   , 1.   , 0.5  , 0.5  , 0.5  , 1.   , 3.   ],
       [4.5  , 1.5  , 0.5  , 0.5  , 0.5  , 1.5  , 4.5  ],
       [6.5  , 2.5  , 0.5  , 0.5  , 0.5  , 2.5  , 6.5  ],
       [9.   , 4.   , 1.   , 0.   , 1.   , 4.   , 9.   ]])

In [11]:
end_time = time.time()
print(end_time - start_time)

0.17325496673583984
